## 행정동 법정동 데이터 매핑

In [138]:
mapping_bh_dong = pd.read_csv('clean_map_dh_dong.csv', index_col=0)
mapping_bh_dong.head()

,시군구명,행정동코드,행정동명,법정동코드,법정동명,법정동개수
0,종로구,11110515,청운효자동,11110101,청운동,10
1,종로구,11110515,청운효자동,11110102,신교동,10
2,종로구,11110515,청운효자동,11110103,궁정동,10
3,종로구,11110515,청운효자동,11110104,효자동,10
4,종로구,11110515,청운효자동,11110105,창성동,10


## 거주인구

In [127]:
import pandas as pd

live_df = pd.read_csv('거주인구_201911.csv', encoding = 'euc-kr')
live_df

,행정구역(동읍면)별,5세별,항목,단위,2019.11 월,Unnamed: 5
0,서울특별시,계,총인구수[명],명,9732577,NaN
1,서울특별시,계,남자인구수[명],명,4746636,NaN
2,서울특별시,계,여자인구수[명],명,4985941,NaN
3,서울특별시,0 - 4세,총인구수[명],명,302041,NaN
4,서울특별시,0 - 4세,남자인구수[명],명,155294,NaN
...,...,...,...,...,...,...
29695,둔촌제2동,95 - 99세,남자인구수[명],명,2,NaN
29696,둔촌제2동,95 - 99세,여자인구수[명],명,23,NaN
29697,둔촌제2동,100+,총인구수[명],명,14,NaN
29698,둔촌제2동,100+,남자인구수[명],명,1,NaN


In [128]:
# len(live_df['행정구역(동읍면)별'].unique())

#### 전처리

In [129]:
# 행정구역 중 '서울특별시' 제거
# 연령 중 '계' 제거
# 컬럼명 변경
# 인덱스,단위,불필요한 컬럼 제거

live_df = live_df[(live_df['행정구역(동읍면)별']!='서울특별시')&(live_df['5세별']!='계')& (live_df['항목']!='총인구수[명]')]
live_df = live_df.reset_index()
live_df = live_df.rename(columns = {'행정구역(동읍면)별':'행정동', '5세별':'연령대','항목':'성별', '2019.11 월':'거주인구수'})
live_df = live_df.drop(['index','단위','Unnamed: 5'], axis = 1)
live_df

,행정동,연령대,성별,거주인구수
0,종로구,0 - 4세,남자인구수[명],1789
1,종로구,0 - 4세,여자인구수[명],1784
2,종로구,5 - 9세,남자인구수[명],2354
3,종로구,5 - 9세,여자인구수[명],2295
4,종로구,10 - 14세,남자인구수[명],2631
...,...,...,...,...
18853,둔촌제2동,90 - 94세,여자인구수[명],66
18854,둔촌제2동,95 - 99세,남자인구수[명],2
18855,둔촌제2동,95 - 99세,여자인구수[명],23
18856,둔촌제2동,100+,남자인구수[명],1


In [130]:
# live_df['연령대'].unique()

In [131]:
# live_df['성별'].unique()

In [132]:
# 연령대: 5세기준 -> 10세 기준
# 성별 표준화: oo인구수[명] -> oo

live_df = live_df.replace(['0 - 4세','5 - 9세'],'10대 미만', regex = True)
live_df = live_df.replace(['10 - 14세', '15 - 19세'],'10대', regex = True)
live_df = live_df.replace(['20 - 24세', '25 - 29세'],'20대', regex = True)
live_df = live_df.replace(['30 - 34세', '35 - 39세'],'30대', regex = True)
live_df = live_df.replace(['40 - 44세', '45 - 49세'],'40대', regex = True)
live_df = live_df.replace(['50 - 54세', '55 - 59세'],'50대', regex = True)
live_df = live_df.replace(['60 - 64세', '65 - 69세'],'60대', regex = True)
live_df = live_df.replace(['70 - 74세', '75 - 79세','80 - 84세', '85 - 89세', '90 - 94세', '95 - 99세', '100+'],'70대 이상', regex = True)
live_df.loc[live_df['연령대']=='70대 이상+','연령대'] ='70대 이상'

live_df['성별'] = live_df['성별'].apply(lambda x: x[:2])
live_df

,행정동,연령대,성별,거주인구수
0,종로구,10대 미만,남자,1789
1,종로구,10대 미만,여자,1784
2,종로구,10대 미만,남자,2354
3,종로구,10대 미만,여자,2295
4,종로구,10대,남자,2631
...,...,...,...,...
18853,둔촌제2동,70대 이상,여자,66
18854,둔촌제2동,70대 이상,남자,2
18855,둔촌제2동,70대 이상,여자,23
18856,둔촌제2동,70대 이상,남자,1


In [133]:
# live_df['연령대'].unique()

In [134]:
# live_df['성별'].unique()

In [135]:
# len(live_df['행정동'].unique())

In [136]:
#행정동 내 구로 되어 있는 행 삭제
## 행정동 내 value가 지역구인 것은 지역구 컬럼에 작성
## 나머지는 앞의 값에 따라 채우는 걸로
## 인덱스 삭제

gu = ['종로구', '중구', '용산구', '성동구', '광진구', '동대문구', '중랑구', '성북구', '강북구',
        '도봉구', '노원구', '은평구', '서대문구', '마포구', '양천구', '강서구', '구로구', '금천구',
        '영등포구', '동작구', '관악구', '서초구', '강남구', '송파구', '강동구']

live_df.loc[live_df['행정동'].isin(gu),'시군구명'] = live_df.loc [live_df['행정동'].isin(gu),'행정동']
live_df = live_df.fillna(method = "ffill")
live_df = live_df[live_df['행정동'].isin(gu) == False].reset_index()
live_df = live_df.drop('index', axis = 1)
live_df

,행정동,연령대,성별,거주인구수,시군구명
0,청운효자동,10대 미만,남자,195,종로구
1,청운효자동,10대 미만,여자,193,종로구
2,청운효자동,10대 미만,남자,264,종로구
3,청운효자동,10대 미만,여자,250,종로구
4,청운효자동,10대,남자,306,종로구
...,...,...,...,...,...
17803,둔촌제2동,70대 이상,여자,66,강동구
17804,둔촌제2동,70대 이상,남자,2,강동구
17805,둔촌제2동,70대 이상,여자,23,강동구
17806,둔촌제2동,70대 이상,남자,1,강동구


In [111]:
# live_df['시군구명'].unique()

In [137]:
#지역, 행정동, 성별, 연령별로 인구수 sum

live_df = live_df[['시군구명','행정동','연령대','성별','거주인구수']].groupby(['시군구명','행정동','성별','연령대']).sum().reset_index()
live_df

,시군구명,행정동,성별,연령대,거주인구수
0,강남구,개포1동,남자,10대,302
1,강남구,개포1동,남자,10대 미만,192
2,강남구,개포1동,남자,20대,434
3,강남구,개포1동,남자,30대,344
4,강남구,개포1동,남자,40대,464
...,...,...,...,...,...
6779,중랑구,중화제2동,여자,30대,1847
6780,중랑구,중화제2동,여자,40대,1761
6781,중랑구,중화제2동,여자,50대,2533
6782,중랑구,중화제2동,여자,60대,1967


#### 연도, 월 컬럼 생성 필요

#### 법정동 행정동 매핑

In [139]:
#유동인구데이터프레임과 행정동법정동매핑데이터 합치기

live_df_b = pd.merge(live_df,mapping_bh_dong)[['시군구명','법정동코드','법정동명','법정동개수','성별','연령대','거주인구수']]
live_df_b

,시군구명,법정동코드,법정동명,법정동개수,성별,연령대,거주인구수
0,강남구,11680107,신사동,2,남자,10대,302
1,강남구,11680110,압구정동,2,남자,10대,302
2,강남구,11680108,논현동,1,남자,10대,302
3,강남구,11680108,논현동,1,남자,10대,302
4,강남구,11680107,신사동,2,남자,10대,302
...,...,...,...,...,...,...,...
205467,중랑구,11260104,묵동,1,여자,70대 이상,1808
205468,중랑구,11260105,망우동,1,여자,70대 이상,1808
205469,중랑구,11260105,망우동,1,여자,70대 이상,1808
205470,중랑구,11260106,신내동,1,여자,70대 이상,1808


In [125]:
# live_df_b[live_df_b['법정동명']=='신사동']

,시군구명,법정동코드,법정동명,법정동개수,성별,연령대,거주인구수
0,강남구,11680107,신사동,2,남자,10대,302
4,강남구,11680107,신사동,2,남자,10대,302
31,강남구,11680107,신사동,2,남자,10대 미만,192
35,강남구,11680107,신사동,2,남자,10대 미만,192
62,강남구,11680107,신사동,2,남자,20대,434
...,...,...,...,...,...,...,...
153964,은평구,11380109,신사동,1,여자,50대,4162
153979,은평구,11380109,신사동,1,여자,60대,3155
153980,은평구,11380109,신사동,1,여자,60대,3155
153995,은평구,11380109,신사동,1,여자,70대 이상,3007


In [157]:
live_df_b = live_df_b.groupby(['시군구명','법정동코드','법정동명','성별','연령대','법정동개수']).sum().reset_index()
live_df_b

,시군구명,법정동코드,법정동명,성별,연령대,법정동개수,거주인구수
0,강남구,11680101,역삼동,남자,10대,1,63336
1,강남구,11680101,역삼동,남자,10대 미만,1,39924
2,강남구,11680101,역삼동,남자,20대,1,73092
3,강남구,11680101,역삼동,남자,30대,1,78416
4,강남구,11680101,역삼동,남자,40대,1,92848
...,...,...,...,...,...,...,...
8587,중랑구,11260106,신내동,여자,30대,1,57522
8588,중랑구,11260106,신내동,여자,40대,1,57668
8589,중랑구,11260106,신내동,여자,50대,1,71920
8590,중랑구,11260106,신내동,여자,60대,1,56140


In [149]:
# len(live_df_b['시군구명'].unique())

25

In [147]:
# len(live_df_b['법정동코드'].unique())

467

In [150]:
# len(live_df_b['법정동명'].unique())

465

In [151]:
# len(live_df_b['성별'].unique())

2

In [152]:
# len(live_df_b['연령대'].unique())

8

In [154]:
# live_df_b['법정동개수'].unique()

array([ 1,  2,  3,  6,  4,  5,  8,  9,  7, 10, 12, 29, 23, 11],
      dtype=int64)

In [165]:
# 거주인구를 법정동 수로 나누기 

live_df_b['거주인구수/n'] = live_df_b['거주인구수']/live_df_b['법정동개수']
live_df_b = live_df_b[['시군구명','법정동코드','법정동명','연령대','거주인구수/n']]
live_df_b

,시군구명,법정동코드,법정동명,연령대,거주인구수/n
0,강남구,11680101,역삼동,10대,63336.0
1,강남구,11680101,역삼동,10대 미만,39924.0
2,강남구,11680101,역삼동,20대,73092.0
3,강남구,11680101,역삼동,30대,78416.0
4,강남구,11680101,역삼동,40대,92848.0
...,...,...,...,...,...
8587,중랑구,11260106,신내동,30대,57522.0
8588,중랑구,11260106,신내동,40대,57668.0
8589,중랑구,11260106,신내동,50대,71920.0
8590,중랑구,11260106,신내동,60대,56140.0


In [ ]:
#저장


## 유동인구

In [174]:
move_df = pd.read_csv('유동인구_201911.csv', index_col = False)
move_df

,기준일ID,시간대구분,행정동코드,총생활인구수,남자0세부터9세생활인구수,남자10세부터14세생활인구수,남자15세부터19세생활인구수,남자20세부터24세생활인구수,남자25세부터29세생활인구수,남자30세부터34세생활인구수,...,여자25세부터29세생활인구수,여자30세부터34세생활인구수,여자35세부터39세생활인구수,여자40세부터44세생활인구수,여자45세부터49세생활인구수,여자50세부터54세생활인구수,여자55세부터59세생활인구수,여자60세부터64세생활인구수,여자65세부터69세생활인구수,여자70세이상생활인구수
0,20191101,0,11110515,15050.3092,538.5561,343.1693,465.5953,638.3558,487.8452,418.2459,...,457.7915,495.0161,662.6627,720.7387,798.4406,609.5900,576.2245,518.4339,461.5705,1161.0233
1,20191101,0,11110530,15262.0768,464.5667,206.4738,208.9448,446.1988,517.4421,568.8790,...,612.8811,607.5549,692.4063,658.8463,774.1853,540.5964,578.8927,553.0628,356.1025,1373.2513
2,20191101,0,11110540,4826.7152,86.4323,66.2275,246.6591,419.1377,152.5504,177.0186,...,124.1948,162.9358,197.8987,195.9057,159.5625,160.1677,193.8269,172.1205,155.1981,442.8197
3,20191101,0,11110550,15734.6611,395.5976,306.7412,446.0090,650.0969,422.4243,398.4986,...,413.9314,471.1692,640.2230,605.8921,818.9924,703.0352,740.8163,588.6950,458.6989,1549.6020
4,20191101,0,11110560,21102.4320,714.3681,516.7259,747.7381,837.7934,554.4978,426.8828,...,527.0677,573.5164,751.0306,707.1281,941.1980,897.7669,1064.3321,909.3537,642.5122,2057.6725
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
305275,20191130,23,11215770,30630.5443,942.6517,460.2612,838.8122,812.1131,1133.3397,1122.4548,...,1261.8670,1262.3717,1263.3378,1082.6724,1152.7335,1312.5082,1440.3064,1261.8346,1050.4839,2395.1408
305276,20191130,23,11590550,20249.7853,718.5044,325.0960,474.2942,566.7332,828.4884,822.7235,...,974.1552,893.9559,878.8802,756.0914,816.4002,810.0740,895.2718,716.6786,507.3680,1680.9896
305277,20191130,23,11680690,18709.2514,639.9231,405.7798,565.2699,412.3526,497.8168,656.1991,...,540.5382,726.8657,790.0580,792.4994,838.7685,760.6855,785.8891,857.9977,597.1024,1261.4907
305278,20191130,23,11470560,18650.4595,598.8156,289.3068,327.2518,453.4583,607.9343,665.4480,...,644.6069,614.2362,734.3827,622.2552,748.7784,751.6004,790.0700,881.8127,677.4338,1455.3024


In [175]:
move_df['행정동코드'].replace([11305610, 11305600, 11305630, 11305606, 11305590, 11305620], [11305615, 11305603, 11305635,  11305608,11305595,11305625], inplace = True)

In [176]:
move_df_clean = move_df.iloc[:,:4]
move_df_clean['남성 10대 미만'] = move_df.iloc[:,4]
move_df_clean['남성 10대 '] = move_df.iloc[:,5]+move_df.iloc[:,6]
move_df_clean['남성 20대 '] = move_df.iloc[:,7]+move_df.iloc[:,8]
move_df_clean['남성 30대 '] = move_df.iloc[:,9]+move_df.iloc[:,10]
move_df_clean['남성 40대 '] = move_df.iloc[:,11]+move_df.iloc[:,12]
move_df_clean['남성 50대 '] = move_df.iloc[:,13]+move_df.iloc[:,14]
move_df_clean['남성 60대 '] = move_df.iloc[:,15]+move_df.iloc[:,16]
move_df_clean['남성 70대 이상'] = move_df.iloc[:,17]

move_df_clean['여성 10대 미만'] = move_df.iloc[:,18]
move_df_clean['여성 10대 '] = move_df.iloc[:,19]+move_df.iloc[:,20]
move_df_clean['여성 20대 '] = move_df.iloc[:,21]+move_df.iloc[:,22]
move_df_clean['여성 30대 '] = move_df.iloc[:,23]+move_df.iloc[:,24]
move_df_clean['여성 40대 '] = move_df.iloc[:,25]+move_df.iloc[:,26]
move_df_clean['여성 50대 '] = move_df.iloc[:,27]+move_df.iloc[:,28]
move_df_clean['여성 60대 '] = move_df.iloc[:,29]+move_df.iloc[:,30]
move_df_clean['여성 70대 이상'] = move_df.iloc[:,31]

move_df_age_mf = move_df_clean.copy()
move_df_age_mf

,기준일ID,시간대구분,행정동코드,총생활인구수,남성 10대 미만,남성 10대,남성 20대,남성 30대,남성 40대,남성 50대,남성 60대,남성 70대 이상,여성 10대 미만,여성 10대,여성 20대,여성 30대,여성 40대,여성 50대,여성 60대,여성 70대 이상
0,20191101,0,11110515,15050.3092,538.5561,808.7646,1126.2010,1009.8517,1232.9844,990.6831,606.3253,738.2129,453.2081,667.3083,874.5138,1157.6788,1519.1793,1185.8145,980.0044,1161.0233
1,20191101,0,11110530,15262.0768,464.5667,415.4186,963.6409,1275.9202,1464.0068,1149.9587,741.9421,825.3937,335.9672,461.0858,1029.2777,1299.9612,1433.0316,1119.4891,909.1653,1373.2513
2,20191101,0,11110540,4826.7152,86.4323,312.8866,571.6881,328.5405,368.5053,331.1288,241.8661,298.6429,88.4599,141.3512,216.7784,360.8345,355.4682,353.9946,327.3186,442.8197
3,20191101,0,11110550,15734.6611,395.5976,752.7502,1072.5212,835.8176,1136.1315,1166.6252,804.9631,966.4363,378.0970,711.2711,937.3254,1111.3922,1424.8845,1443.8515,1047.3939,1549.6020
4,20191101,0,11110560,21102.4320,714.3681,1264.4640,1392.2912,993.1212,1388.7617,1651.7577,1201.2615,1153.8036,620.2154,1031.5703,1146.3068,1324.5470,1648.3261,1962.0990,1551.8659,2057.6725
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
305275,20191130,23,11215770,30630.5443,942.6517,1299.0734,1945.4528,2247.9428,2214.3790,2359.9383,1720.3409,1512.3150,808.9075,1088.9364,2269.2179,2525.7095,2235.4059,2752.8146,2312.3185,2395.1408
305276,20191130,23,11590550,20249.7853,718.5044,799.3902,1395.2216,1762.8462,1440.7999,1280.8830,958.5453,861.9457,607.5501,724.3524,1744.0380,1772.8361,1572.4916,1705.3458,1224.0466,1680.9896
305277,20191130,23,11680690,18709.2514,639.9231,971.0497,910.1694,1348.8054,1636.8279,1477.8911,1008.4476,803.1843,498.4764,906.4359,1096.6821,1516.9237,1631.2679,1546.5746,1455.1001,1261.4907
305278,20191130,23,11470560,18650.4595,598.8156,616.5586,1061.3926,1448.0654,1503.7019,1587.4422,1139.3453,901.3641,657.4836,702.0504,1158.3677,1348.6189,1371.0336,1541.6704,1559.2465,1455.3024


In [177]:
move_df_age = move_df_age_mf.iloc[:,:4]

move_df_age['10대 미만'] = move_df_age_mf.iloc[:,4]+move_df_age_mf.iloc[:,12]
move_df_age['10대'] = move_df_age_mf.iloc[:,5]+move_df_age_mf.iloc[:,13]
move_df_age['20대'] = move_df_age_mf.iloc[:,6]+move_df_age_mf.iloc[:,14]
move_df_age['30대'] = move_df_age_mf.iloc[:,7]+move_df_age_mf.iloc[:,15]
move_df_age['40대'] = move_df_age_mf.iloc[:,8]+move_df_age_mf.iloc[:,16]
move_df_age['50대'] = move_df_age_mf.iloc[:,9]+move_df_age_mf.iloc[:,17]
move_df_age['60대'] = move_df_age_mf.iloc[:,10]+move_df_age_mf.iloc[:,18]
move_df_age['70대 이상'] = move_df_age_mf.iloc[:,11]+move_df_age_mf.iloc[:,19]

In [178]:
move_df_age

,기준일ID,시간대구분,행정동코드,총생활인구수,10대 미만,10대,20대,30대,40대,50대,60대,70대 이상
0,20191101,0,11110515,15050.3092,991.7642,1476.0729,2000.7148,2167.5305,2752.1637,2176.4976,1586.3297,1899.2362
1,20191101,0,11110530,15262.0768,800.5339,876.5044,1992.9186,2575.8814,2897.0384,2269.4478,1651.1074,2198.6450
2,20191101,0,11110540,4826.7152,174.8922,454.2378,788.4665,689.3750,723.9735,685.1234,569.1847,741.4626
3,20191101,0,11110550,15734.6611,773.6946,1464.0213,2009.8466,1947.2098,2561.0160,2610.4767,1852.3570,2516.0383
4,20191101,0,11110560,21102.4320,1334.5835,2296.0343,2538.5980,2317.6682,3037.0878,3613.8567,2753.1274,3211.4761
...,...,...,...,...,...,...,...,...,...,...,...,...
305275,20191130,23,11215770,30630.5443,1751.5592,2388.0098,4214.6707,4773.6523,4449.7849,5112.7529,4032.6594,3907.4558
305276,20191130,23,11590550,20249.7853,1326.0545,1523.7426,3139.2596,3535.6823,3013.2915,2986.2288,2182.5919,2542.9353
305277,20191130,23,11680690,18709.2514,1138.3995,1877.4856,2006.8515,2865.7291,3268.0958,3024.4657,2463.5477,2064.6750
305278,20191130,23,11470560,18650.4595,1256.2992,1318.6090,2219.7603,2796.6843,2874.7355,3129.1126,2698.5918,2356.6665
